# Sentiment Analysis with an RNN

In this notebook, you'll implement a recurrent neural network that performs sentiment analysis. 
>Using an RNN rather than a strictly feedforward network is more accurate since we can include information about the *sequence* of words. 

Here we'll use a dataset of movie reviews, accompanied by sentiment labels: positive or negative.

<img src="assets/reviews_ex.png" width=40%>

### Network Architecture

The architecture for this network is shown below.

<img src="assets/network_diagram.png" width=40%>

>**First, we'll pass in words to an embedding layer.** We need an embedding layer because we have tens of thousands of words, so we'll need a more efficient representation for our input data than one-hot encoded vectors. You should have seen this before from the Word2Vec lesson. You can actually train an embedding with the Skip-gram Word2Vec model and use those embeddings as input, here. However, it's good enough to just have an embedding layer and let the network learn a different embedding table on its own. *In this case, the embedding layer is for dimensionality reduction, rather than for learning semantic representations.*

>**After input words are passed to an embedding layer, the new embeddings will be passed to LSTM cells.** The LSTM cells will add *recurrent* connections to the network and give us the ability to include information about the *sequence* of words in the movie review data. 

>**Finally, the LSTM outputs will go to a sigmoid output layer.** We're using a sigmoid function because positive and negative = 1 and 0, respectively, and a sigmoid will output predicted, sentiment values between 0-1. 

We don't care about the sigmoid outputs except for the **very last one**; we can ignore the rest. We'll calculate the loss by comparing the output at the last time step and the training label (pos or neg).

---
### Load in and visualize the data

In [1]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [2]:
print(reviews[:2000])
print()
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

## Data pre-processing

The first step when building a neural network model is getting your data into the proper form to feed into the network. Since we're using embedding layers, we'll need to encode each word with an integer. We'll also want to clean it up a bit.

You can see an example of the reviews data above. Here are the processing steps, we'll want to take:
>* We'll want to get rid of periods and extraneous punctuation.
* Also, you might notice that the reviews are delimited with newline characters `\n`. To deal with those, I'm going to split the text into each review using `\n` as the delimiter. 
* Then I can combined all the reviews back together into one big string.

First, let's remove all punctuation. Then get all the text without the newlines and split it into individual words.

In [3]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [6]:
[c for c in reviews if c not in punctuation]

['b',
 'r',
 'o',
 'm',
 'w',
 'e',
 'l',
 'l',
 ' ',
 'h',
 'i',
 'g',
 'h',
 ' ',
 'i',
 's',
 ' ',
 'a',
 ' ',
 'c',
 'a',
 'r',
 't',
 'o',
 'o',
 'n',
 ' ',
 'c',
 'o',
 'm',
 'e',
 'd',
 'y',
 ' ',
 ' ',
 'i',
 't',
 ' ',
 'r',
 'a',
 'n',
 ' ',
 'a',
 't',
 ' ',
 't',
 'h',
 'e',
 ' ',
 's',
 'a',
 'm',
 'e',
 ' ',
 't',
 'i',
 'm',
 'e',
 ' ',
 'a',
 's',
 ' ',
 's',
 'o',
 'm',
 'e',
 ' ',
 'o',
 't',
 'h',
 'e',
 'r',
 ' ',
 'p',
 'r',
 'o',
 'g',
 'r',
 'a',
 'm',
 's',
 ' ',
 'a',
 'b',
 'o',
 'u',
 't',
 ' ',
 's',
 'c',
 'h',
 'o',
 'o',
 'l',
 ' ',
 'l',
 'i',
 'f',
 'e',
 ' ',
 ' ',
 's',
 'u',
 'c',
 'h',
 ' ',
 'a',
 's',
 ' ',
 ' ',
 't',
 'e',
 'a',
 'c',
 'h',
 'e',
 'r',
 's',
 ' ',
 ' ',
 ' ',
 'm',
 'y',
 ' ',
 ' ',
 ' ',
 'y',
 'e',
 'a',
 'r',
 's',
 ' ',
 'i',
 'n',
 ' ',
 't',
 'h',
 'e',
 ' ',
 't',
 'e',
 'a',
 'c',
 'h',
 'i',
 'n',
 'g',
 ' ',
 'p',
 'r',
 'o',
 'f',
 'e',
 's',
 's',
 'i',
 'o',
 'n',
 ' ',
 'l',
 'e',
 'a',
 'd',
 ' ',
 'm',
 'e',
 ' '

In [4]:
# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

In [5]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

### Encoding the words

The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

> **Exercise:** Now you're going to encode the words with integers. Build a dictionary that maps words to integers. Later we're going to pad our input vectors with zeros, so make sure the integers **start at 1, not 0**.
> Also, convert the reviews to integers and store the reviews in a new list called `reviews_ints`. 

In [17]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
all_uniq_words = set(words)
vocab_to_int = {}
for index, uniq_word in enumerate(all_uniq_words):
    vocab_to_int[uniq_word] = index + 1

In [30]:
print(type(vocab_to_int))
print(list(vocab_to_int.items())[:10])

<class 'dict'>
[('soderberg', 1), ('weaker', 2), ('fleed', 3), ('mornell', 4), ('effeil', 5), ('glamourise', 6), ('lobe', 7), ('wilona', 8), ('mtro', 9), ('decrees', 10)]


In [71]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
review_ints = []

reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

**Test your code**

As a text that you've implemented the dictionary correctly, print out the number of unique words in your vocabulary and the contents of the first, tokenized review.

In [72]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[62957, 52509, 66541, 64695, 70607, 58838, 3389, 45252, 34669, 27715, 37282, 29908, 44231, 37329, 43253, 11516, 50286, 672, 62598, 46859, 44231, 51051, 72941, 55349, 4313, 27715, 67699, 60058, 22933, 20898, 8818, 60219, 51704, 62957, 52509, 53859, 46719, 66541, 11310, 20527, 8818, 10763, 64877, 66541, 51051, 27715, 70291, 8818, 71952, 1309, 27715, 27689, 60484, 2162, 38306, 49658, 71997, 59010, 2164, 48708, 51051, 23904, 27715, 17256, 36244, 27715, 71076, 10774, 29169, 10137, 20898, 36244, 27715, 4438, 45088, 35868, 53361, 2164, 60484, 34895, 45088, 65711, 27715, 45520, 4313, 66369, 64695, 1065, 28677, 42773, 8818, 3521, 35023, 27715, 672, 45088, 60235, 23040, 34669, 52509, 64695, 19440, 49808, 34675, 45088, 20851, 60705, 8818, 64709, 8944, 36244, 3060, 51051, 1065, 24450, 8818, 62957, 52509, 45088, 25031, 51704, 57018, 38766, 36244, 72941, 73192, 14924, 51704, 62957, 52509, 66541, 61857, 74035, 3003, 64695, 62366, 51704, 3389, 57471, 73308]]


### Encoding the labels

Our labels are "positive" or "negative". To use these labels in our network, we need to convert them to 0 and 1.

> **Exercise:** Convert labels from `positive` and `negative` to 1 and 0, respectively, and place those in a new list, `encoded_labels`.

In [73]:
# 1=positive, 0=negative label conversion
encoded_labels = []
for lab in labels.split():
    if lab == 'positive':
        encoded_labels.append(1)
    elif lab == 'negative':
        encoded_labels.append(0)
        
# encoded_labels

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,


### Removing Outliers

As an additional pre-processing step, we want to make sure that our reviews are in good shape for standard processing. That is, our network will expect a standard input text size, and so, we'll want to shape our reviews into a specific length. We'll approach this task in two main steps:

1. Getting rid of extremely long or short reviews; the outliers
2. Padding/truncating the remaining data so that we have reviews of the same length.

<img src="assets/outliers_padding_ex.png" width=40%>

Before we pad our review text, we should check for reviews of extremely short or long lengths; outliers that may mess with our training.

In [57]:
[len(x) for x in reviews_ints]

[140,
 114,
 447,
 786,
 150,
 146,
 124,
 153,
 121,
 401,
 184,
 104,
 112,
 331,
 346,
 159,
 451,
 68,
 330,
 295,
 290,
 347,
 86,
 148,
 283,
 105,
 231,
 144,
 148,
 180,
 162,
 140,
 196,
 67,
 333,
 87,
 132,
 134,
 155,
 181,
 219,
 20,
 581,
 541,
 177,
 187,
 186,
 178,
 144,
 121,
 106,
 860,
 109,
 227,
 317,
 198,
 150,
 508,
 145,
 243,
 246,
 127,
 122,
 331,
 166,
 315,
 168,
 156,
 451,
 519,
 412,
 346,
 56,
 238,
 305,
 157,
 221,
 254,
 338,
 118,
 359,
 196,
 167,
 118,
 551,
 117,
 248,
 233,
 111,
 353,
 337,
 262,
 991,
 729,
 566,
 220,
 89,
 398,
 1853,
 116,
 460,
 132,
 549,
 157,
 185,
 166,
 553,
 697,
 207,
 187,
 736,
 249,
 185,
 174,
 163,
 197,
 57,
 197,
 280,
 171,
 98,
 164,
 184,
 355,
 134,
 194,
 223,
 994,
 327,
 137,
 168,
 174,
 110,
 161,
 225,
 115,
 111,
 140,
 96,
 210,
 126,
 138,
 196,
 143,
 297,
 54,
 328,
 146,
 43,
 1037,
 149,
 166,
 109,
 345,
 339,
 461,
 76,
 145,
 230,
 325,
 119,
 252,
 262,
 367,
 117,
 54,
 117,
 195,
 55,

In [63]:
Counter([len(x) for x in reviews_ints])

Counter({140: 156,
         114: 110,
         447: 16,
         786: 3,
         150: 124,
         146: 130,
         124: 167,
         153: 126,
         121: 149,
         401: 13,
         184: 82,
         104: 54,
         112: 93,
         331: 33,
         346: 29,
         159: 131,
         451: 21,
         68: 54,
         330: 23,
         295: 33,
         290: 30,
         347: 25,
         86: 44,
         148: 151,
         283: 27,
         105: 65,
         231: 44,
         144: 142,
         180: 92,
         162: 126,
         196: 69,
         67: 42,
         333: 28,
         87: 42,
         132: 185,
         134: 160,
         155: 122,
         181: 75,
         219: 55,
         20: 3,
         581: 5,
         541: 11,
         177: 94,
         187: 91,
         186: 90,
         178: 78,
         106: 72,
         860: 4,
         109: 87,
         227: 46,
         317: 30,
         198: 76,
         508: 13,
         145: 131,
         243: 47,
    

In [74]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


Okay, a couple issues here. We seem to have one review with zero length. And, the maximum review length is way too many steps for our RNN. We'll have to remove any super short reviews and truncate super long reviews. This removes outliers and should allow our model to train more efficiently.

> **Exercise:** First, remove *any* reviews with zero length from the `reviews_ints` list and their corresponding label in `encoded_labels`.

In [65]:
reviews_ints[0]

[18725,
 36244,
 64695,
 13002,
 2162,
 27171,
 2688,
 24236,
 32033,
 64695,
 64347,
 43721,
 8164,
 21379,
 64695,
 34250,
 16787,
 51704,
 66541,
 64695,
 21979,
 46029,
 36244,
 36766,
 58838,
 64695,
 45159,
 70055,
 27021,
 66541,
 65052,
 73325,
 68207,
 54144,
 40936,
 42712,
 23383,
 27715,
 55939,
 9743,
 36244,
 3389,
 53859,
 23744,
 4135,
 3389,
 40195,
 36766,
 27715,
 71076,
 29908,
 21379,
 32788,
 48722,
 34401,
 22733,
 23966,
 3389,
 24879,
 21992,
 17043,
 9384,
 29291,
 39012,
 47897,
 27715,
 1281,
 62488,
 14236,
 65052,
 17043,
 27715,
 64607,
 24005,
 32380,
 57133,
 61905,
 10756,
 70277,
 8818,
 64695,
 68439,
 64633,
 45589,
 64695,
 30086,
 42812,
 3389,
 53859,
 34530,
 64877,
 56636,
 54151,
 14924,
 21379,
 37329,
 16404,
 17931,
 23383,
 62252,
 21744,
 7203,
 4082,
 62252,
 38050,
 62897,
 73388,
 53361,
 26686,
 56539,
 38306,
 14236,
 64344,
 1118]

In [66]:
encoded_labels[0]

1

In [75]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.
temp_reviews_ints = []
temp_encoded_labels = []

for index, sentence in enumerate(reviews_ints):
    if len(sentence) > 0:
        temp_reviews_ints.append(reviews_ints[index])
        temp_encoded_labels.append(encoded_labels[index])

reviews_ints = temp_reviews_ints
encoded_labels = temp_encoded_labels

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 200.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [92]:
list(np.zeros(3, dtype = 'int')) + list([1,2])

# np.concatenate(np.zeros(3),[1,2])

[0, 0, 0, 1, 2]

In [88]:
np.zeros(3)

array([0., 0., 0.])

In [90]:
np.zeros(3).reshape(-1).shape

(3,)

In [119]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## implement function
    features = []
    
    for review in reviews_ints:
        if len(review) >= 200:
            temp_review = review[:200]
        else:
            temp_review = list(np.zeros(200 - len(review), dtype = 'int')) + review
    
        features.append(temp_review)
    
    return np.array(features)

In [120]:
# Test your implementation!

seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)

In [121]:
type(features)

numpy.ndarray

In [122]:
len(features)

25000

In [123]:
features[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 62957, 52509, 66541,
        64695, 70607, 58838,  3389, 45252, 34669, 27715, 37282, 29908,
        44231, 37329, 43253, 11516, 50286,   672, 62598, 46859, 44231,
        51051, 72941, 55349,  4313, 27715, 67699, 60058, 22933, 20898,
         8818, 60219, 51704, 62957, 52509, 53859, 46719, 66541, 11310,
        20527,  8818, 10763, 64877, 66541, 51051, 27715, 70291,  8818,
        71952,  1309, 27715, 27689, 60484,  2162, 38306, 49658, 71997,
        59010,  2164, 48708, 51051, 23904, 27715, 17256, 36244, 27715,
      

In [124]:
## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [24898 61901 36032 44231 15364 54729 54176 27171 68267 68207]
 [71327 43721 44231 64695  3613 13115 73926 14350 66541 39967]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [34895 45088  9964 37031 72941 48285 68933 20898 49850  8818]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [27715 28712 31781 23383 64695 13115 49474  8891  4313  6045]
 [ 4313  1994 22038 47524  3861 45514 13433 45589 20042 59853]
 [    0     0     0     0     0     0     0     0     0

## Training, Validation, Test

With our data in nice shape, we'll split it into training, validation, and test sets.

> **Exercise:** Create the training, validation, and test sets. 
* You'll need to create sets for the features and the labels, `train_x` and `train_y`, for example. 
* Define a split fraction, `split_frac` as the fraction of data to **keep** in the training set. Usually this is set to 0.8 or 0.9. 
* Whatever data is left will be split in half to create the validation and *testing* data.

In [138]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
train_length = int(np.round(split_frac * len(features)))
test_valid_length = int((len(features) - train_length)/2)

In [146]:
## print out the shapes of your resultant feature data
encoded_labels = np.array(encoded_labels)

train_x = features[:train_length]
train_y = encoded_labels[:train_length]

val_x = features[train_length:train_length + test_valid_length]
val_y = encoded_labels[train_length:train_length + test_valid_length]

test_x = features[train_length + test_valid_length:]
test_y = encoded_labels[train_length + test_valid_length:]

In [147]:
print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

(20000, 200)
(2500, 200)
(2500, 200)


**Check your work**

With train, validation, and test fractions equal to 0.8, 0.1, 0.1, respectively, the final, feature data shapes should look like:
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

---
## DataLoaders and Batching

After creating training, test, and validation data, we can create DataLoaders for this data by following two steps:
1. Create a known format for accessing our data, using [TensorDataset](https://pytorch.org/docs/stable/data.html#) which takes in an input set of data and a target set of data with the same first dimension, and creates a dataset.
2. Create DataLoaders and batch our training, validation, and test Tensor datasets.

```
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, batch_size=batch_size)
```

This is an alternative to creating a generator function for batching our data into full batches.

In [149]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [150]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[45088, 61822,  3003,  ..., 72785, 27715, 59525],
        [70649, 16024, 70649,  ..., 37329,   255, 54461],
        [    0,     0,     0,  ..., 22434, 43174, 50365],
        ...,
        [    0,     0,     0,  ..., 73308, 12543, 16404],
        [    0,     0,     0,  ..., 49973, 21992, 58365],
        [67177, 51238, 64695,  ..., 38306, 24090, 50286]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 0], dtype=torch.int32)


---
# Sentiment Network with PyTorch

Below is where you'll define the network.

<img src="assets/network_diagram.png" width=40%>

The layers are as follows:
1. An [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) that converts our word tokens (integers) into embeddings of a specific size.
2. An [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) defined by a hidden_state size and number of layers
3. A fully-connected output layer that maps the LSTM layer outputs to a desired output_size
4. A sigmoid activation layer which turns all outputs into a value 0-1; return **only the last sigmoid output** as the output of this network.

### The Embedding Layer

We need to add an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) because there are 74000+ words in our vocabulary. It is massively inefficient to one-hot encode that many classes. So, instead of one-hot encoding, we can have an embedding layer and use that layer as a lookup table. You could train an embedding layer using Word2Vec, then load it here. But, it's fine to just make a new layer, using it for only dimensionality reduction, and let the network learn the weights.


### The LSTM Layer(s)

We'll create an [LSTM](https://pytorch.org/docs/stable/nn.html#lstm) to use in our recurrent network, which takes in an input_size, a hidden_dim, a number of layers, a dropout probability (for dropout between multiple layers), and a batch_first parameter.

Most of the time, you're network will have better performance with more layers; between 2-3. Adding more layers allows the network to learn really complex relationships. 

> **Exercise:** Complete the `__init__`, `forward`, and `init_hidden` functions for the SentimentRNN model class.

Note: `init_hidden` should initialize the hidden and cell state of an lstm layer to all zeros, and move those state to GPU, if available.

In [151]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [152]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define all layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
                
        return hidden
        

## Instantiate the network

Here, we'll instantiate the network. First up, defining the hyperparameters.

* `vocab_size`: Size of our vocabulary or the range of values for our input, word tokens.
* `output_size`: Size of our desired output; the number of class scores we want to output (pos/neg).
* `embedding_dim`: Number of columns in the embedding lookup table; size of our embeddings.
* `hidden_dim`: Number of units in the hidden layers of our LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `n_layers`: Number of LSTM layers in the network. Typically between 1-3

> **Exercise:** Define the model  hyperparameters.


In [153]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


---
## Training

Below is the typical training code. If you want to do this yourself, feel free to delete all this code and implement it yourself. You can also add code to save a model by name.

>We'll also be using a new kind of cross entropy loss, which is designed to work with a single Sigmoid output. [BCELoss](https://pytorch.org/docs/stable/nn.html#bceloss), or **Binary Cross Entropy Loss**, applies cross entropy loss to a single value between 0 and 1.

We also have some data and training hyparameters:

* `lr`: Learning rate for our optimizer.
* `epochs`: Number of times to iterate through the training dataset.
* `clip`: The maximum gradient value to clip at (to prevent exploding gradients).

In [154]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [155]:
# training params

epochs = 4 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/4... Step: 100... Loss: 0.670748... Val Loss: 0.630902
Epoch: 1/4... Step: 200... Loss: 0.592679... Val Loss: 0.619576
Epoch: 1/4... Step: 300... Loss: 0.603898... Val Loss: 0.551590
Epoch: 1/4... Step: 400... Loss: 0.538335... Val Loss: 0.530907
Epoch: 2/4... Step: 500... Loss: 0.496077... Val Loss: 0.531365
Epoch: 2/4... Step: 600... Loss: 0.308648... Val Loss: 0.484327
Epoch: 2/4... Step: 700... Loss: 0.471965... Val Loss: 0.532015
Epoch: 2/4... Step: 800... Loss: 0.332950... Val Loss: 0.439493
Epoch: 3/4... Step: 900... Loss: 0.203692... Val Loss: 0.417565
Epoch: 3/4... Step: 1000... Loss: 0.293747... Val Loss: 0.497547
Epoch: 3/4... Step: 1100... Loss: 0.263601... Val Loss: 0.458431
Epoch: 3/4... Step: 1200... Loss: 0.258189... Val Loss: 0.427846
Epoch: 4/4... Step: 1300... Loss: 0.212632... Val Loss: 0.503504
Epoch: 4/4... Step: 1400... Loss: 0.357216... Val Loss: 0.520202
Epoch: 4/4... Step: 1500... Loss: 0.221282... Val Loss: 0.498614
Epoch: 4/4... Step: 1600... Loss: 

---
## Testing

There are a few ways to test your network.

* **Test data performance:** First, we'll see how our trained model performs on all of our defined test_data, above. We'll calculate the average loss and accuracy over the test data.

* **Inference on user-generated data:** Second, we'll see if we can input just one example review at a time (without a label), and see what the trained model predicts. Looking at new, user input data like this, and predicting an output label, is called **inference**.

In [156]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.522
Test accuracy: 0.794


### Inference on a test review

You can change this test_review to any text that you want. Read it and think: is it pos or neg? Then see if your model predicts correctly!
    
> **Exercise:** Write a `predict` function that takes in a trained net, a plain text_review, and a sequence length, and prints out a custom statement for a positive or negative review!
* You can use any functions that you've already defined or define any helper functions you want to complete `predict`, but it should just take in a trained net, a text review, and a sequence length.


In [157]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

In [159]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_neg)
print(test_ints)

[[27715, 59559, 14114, 45088, 64126, 64344, 63773, 9964, 4483, 53361, 45088, 27562, 72941, 43220, 66306, 1994, 14114, 27136, 66388, 63773, 53361, 27715, 24005, 9964, 44523]]


In [160]:
# test sequence padding
seq_length=200
features = pad_features(test_ints, seq_length)

print(features)

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [164]:
def predict(net, test_review, sequence_length=200):
    ''' Prints out whether a give review is predicted to be 
        positive or negative in sentiment, using a trained model.
        
        params:
        net - A trained net 
        test_review - a review made of normal text and punctuation
        sequence_length - the padded length of a review
        '''
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    
    # print custom response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")   
    
    # print custom response based on whether test_review is pos/neg

In [165]:
# positive test review
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [166]:
# call function
# try negative and positive reviews!
seq_length=200
predict(net, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.004100
Negative review detected.


### Try out test_reviews of your own!

Now that you have a trained model and a predict function, you can pass in _any_ kind of text and this model will predict whether the text has a positive or negative sentiment. Push this model to its limits and try to find what words it associates with positive or negative.

Later, you'll learn how to deploy a model like this to a production environment so that it can respond to any kind of user data put into a web app!